In [ ]:
# =========================================================
#  プロジェクトルートを sys.path に追加
# =========================================================
import sys
import os

PROJECT_ROOT = os.path.abspath("..")  # notebooks/ から見て1つ上
sys.path.append(PROJECT_ROOT)

print("Added to sys.path:", PROJECT_ROOT)


Added to sys.path: c:\Users\AVRJ0006\YamatoDX\YAMATO_learning_practice\my_time_series_benchmark\project_git\My-Time-Series-Benchmark\mtsb


In [ ]:
# =========================================================
#  インポート（naive.ipynb と完全一致）
# =========================================================
import pandas as pd
import numpy as np
from pathlib import Path

# モデル
from models.baseline.drift import DriftModel

# 指標
from evaluation.basic.mae import MAE
from evaluation.basic.rmse import RMSE
from evaluation.basic.mape import MAPE
from evaluation.basic.smape import SMAPE
from evaluation.basic.r2 import R2


In [ ]:
# =========================================================
#  データ読み込み（パスも naive.ipynb と完全一致）
# =========================================================
df = pd.read_csv("../data/raw/daily_sales.csv")
df["date"] = pd.to_datetime(df["date"])
df = df.set_index("date")

y = df["sales"]

horizon = 30

train = y.iloc[:-horizon]
test = y.iloc[-horizon:]

train.head(), test.head()


(date
 2023-01-01    127.94
 2023-01-02    133.14
 2023-01-03    129.73
 2023-01-04    122.88
 2023-01-05    124.41
 Name: sales, dtype: float64,
 date
 2023-06-20    133.05
 2023-06-21    134.51
 2023-06-22    125.49
 2023-06-23    130.29
 2023-06-24    133.10
 Name: sales, dtype: float64)

In [ ]:
# =========================================================
#  データ読み込み（パスも naive.ipynb と完全一致）
# =========================================================
df = pd.read_csv("../data/raw/daily_sales.csv")
df["date"] = pd.to_datetime(df["date"])
df = df.set_index("date")

y = df["sales"]

horizon = 30

train = y.iloc[:-horizon]
test = y.iloc[-horizon:]

train.head(), test.head()


(date
 2023-01-01    127.94
 2023-01-02    133.14
 2023-01-03    129.73
 2023-01-04    122.88
 2023-01-05    124.41
 Name: sales, dtype: float64,
 date
 2023-06-20    133.05
 2023-06-21    134.51
 2023-06-22    125.49
 2023-06-23    130.29
 2023-06-24    133.10
 Name: sales, dtype: float64)

In [ ]:
# =========================================================
#  DriftModel の学習 & 予測
# =========================================================
model = DriftModel(freq="D")
model.fit(train)

y_pred = model.forecast(horizon)
y_pred.head()


2023-06-20    128.694438
2023-06-21    128.698876
2023-06-22    128.703314
2023-06-23    128.707751
2023-06-24    128.712189
Freq: D, Name: drift, dtype: float64

In [ ]:
# =========================================================
#  評価指標（順番も naive.ipynb と一致）
# =========================================================
metrics = {
    "mae": MAE().compute(test, y_pred),
    "rmse": RMSE().compute(test, y_pred),
    "mape": MAPE().compute(test, y_pred),
    "smape": SMAPE().compute(test, y_pred),
    "r2": R2().compute(test, y_pred),
}

metrics


{'mae': 13.646743589743592,
 'rmse': 17.113338768216316,
 'mape': 9.147068696544357,
 'smape': 9.809876844648118,
 'r2': -1.2189519242965106}

In [ ]:
# =========================================================
#  horizon 別誤差（naive.ipynb のセル6に完全一致）
# =========================================================
horizon_rows = []
for h in range(1, horizon + 1):
    yt = test.iloc[h-1]
    yp = y_pred.iloc[h-1]

    horizon_rows.append({
        "horizon": h,
        "abs_error": abs(yt - yp),
        "squared_error": (yt - yp)**2,
        "pct_error": abs(yt - yp)/abs(yt) * 100 if yt != 0 else None,
    })

horizon_df = pd.DataFrame(horizon_rows)
horizon_df.head()


,horizon,abs_error,squared_error,pct_error
0,1,4.355562,18.970921,3.273628
1,2,5.811124,33.769165,4.320217
2,3,3.213314,10.325384,2.560613
3,4,1.582249,2.503510,1.214405
4,5,4.387811,19.252882,3.296627


In [ ]:
# =========================================================
#  CSV 出力（naive.ipynb の出力方式に完全一致）
# =========================================================

# forecast
forecast_df = pd.DataFrame({
    "date": y_pred.index,
    "actual": test.values,
    "forecast": y_pred.values,
    "model": "drift",
})
forecast_df.to_csv("../../exports_for_tableau/drift_forecast.csv", index=False)

# metrics
metrics_df = pd.DataFrame([{
    "model": "drift",
    **metrics
}])
metrics_df.to_csv("../../exports_for_tableau/drift_metrics.csv", index=False)

# horizon error
horizon_df.to_csv("../../exports_for_tableau/drift_horizon_error.csv", index=False)

print("CSV files exported successfully!")


CSV files exported successfully!
